In [2]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [3]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [4]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [5]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [6]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [7]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [8]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df


In [9]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권_상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [10]:
import pandas as pd

df = load_to_df('상권-상권변화지표')
# df12 = load_to_df('상권-직장인구')
df9 = df.copy()

df = load_to_df('상권-점포')
# df12 = load_to_df('상권-직장인구')
df18 = df.copy()

In [11]:
d1=df9.loc[df9['기준_년_분기_코드'].str.contains('2019_1', na=False) ]
d2=df9.loc[df9['기준_년_분기_코드'].str.contains('2019_2', na=False) ]

In [12]:
df9.head()

,기준_년_분기_코드,상권_코드,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2020_1,1001496,HH,정체,118,61,110,54
1,2020_1,1001495,LL,다이나믹,103,53,110,54
2,2020_1,1001494,HH,정체,147,71,110,54
3,2020_1,1001493,HH,정체,119,61,110,54
4,2020_1,1001492,HH,정체,147,62,110,54


In [12]:
# 운영영업기간 > 서울평균운영영업기간, 폐업영업기간 > 서울평균폐업영업기간 (정체) HH
# 운영영업기간 > 서울평균운영영업기간, 폐업영업기간 < 서울평균폐업영업기간 (상권축소) HL
# 운영영업기간 < 서울평균운영영업기간, 폐업영업기간 > 서울평균폐업영업기간 (상권확장) LH
# 운영영업기간 < 서울평균운영영업기간, 폐업영업기간 < 서울평균폐업영업기간 (다이나믹) LL

In [13]:
# def point()
# df9.loc[ (df9['운영_영업_개월_평균'] >= df9['서울_운영_영업_개월_평균']) & (df9['폐업_영업_개월_평균'] >= df9['서울_폐업_영업_개월_평균']), "point"]='HH 정체'
# df9.loc[ (df9['운영_영업_개월_평균'] >= df9['서울_운영_영업_개월_평균']) & (df9['폐업_영업_개월_평균'] <= df9['서울_폐업_영업_개월_평균']), "point"]='HL 상권축소'
# df9.loc[ (df9['운영_영업_개월_평균'] <= df9['서울_운영_영업_개월_평균']) & (df9['폐업_영업_개월_평균'] >= df9['서울_폐업_영업_개월_평균']), "point"]='LH 상권확장'
# df9.loc[ (df9['운영_영업_개월_평균'] <= df9['서울_운영_영업_개월_평균']) & (df9['폐업_영업_개월_평균'] <= df9['서울_폐업_영업_개월_평균']), "point"]='LL 다이나믹'

In [14]:
# df9.head(15)

In [15]:
# d1[d1['상권_변화_지표']=='HH']

In [16]:
# df9.loc[(df9['기준_년_분기_코드'] == '2019_1') & (df9['상권_코드'] == '1000001')]

In [23]:
def one_hot_encoding(train_data): 
    for cate in select_column: 
        #ONE HOT ENCODING 처리 
        prefix = cate  #붙여둘 단어를 value 자체로함. 이건 다르게 설정 가능 
        one_hot_encoding = pd.get_dummies(train_data[[cate]], prefix=prefix) #OHE 진행 
        train_data = pd.concat([pd.DataFrame(train_data), one_hot_encoding],axis=1) #기존 데이터프레임에 합쳐주기 
        train_data = train_data.drop(cate,axis=1) #ONE HOT ENCODING 대상이었던 칼럼은 삭제 
        return train_data.copy()

In [13]:
# !conda info --envs

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [36]:
ohe = OneHotEncoder() #[0] 번째 컬럼 Encoding
ohe.fit(df9[['상권_변화_지표']].values)
result = ohe.transform(df9[['상권_변화_지표']].values)
col=ohe.categories_
# print(result)
print(col)
result.toarray()

[array(['HH', 'HL', 'LH', 'LL'], dtype=object)]


array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [38]:
# df9.get_dummies(df9['상권_변화_지표'])

In [ ]:
# pd.get_dummies(d_t_train)
dummies = pd.get_dummies(d_t_train[['Sex', 'Embarked', 'AgeGroup']])

merged_t_train = d_t_train.merge(dummies,
            left_index=True, right_index=True)
merged_t_train